## パイプライン

### パイプラインとは
#### 変換器(特徴量の生成と選択)と予測気を直列に繋げ，一連の処理をまとめて実行できるようにする仕組みのことをパイプラインと言う．
### メリット
1. いくつかの前処理を実行している場合でも，fit や predict をパイプラインに対して1度だけ呼ぶだけで良い
2. パイプラインに対して，グリッドサーチを行う際，パラメータを1度に指定できる．fit を読んだ後の変換器をキャッシュでき，性能向上が期待できる．
3. transformer と estimatorに同じデータが使われるため，交差検証の際にデータがリークされることを防げる

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import pickle

from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_selection import SelectFromModel

# 変換器
from sklearn.preprocessing import StandardScaler

# 予測器
import lightgbm as lgb

# パイプライン
from sklearn.pipeline import Pipeline

# パラメーター調整
import optuna
import optuna.integration.lightgbm as lgb_tune
from lightgbm import early_stopping

# 評価関数
from sklearn.metrics import mean_squared_error, r2_score

# ベイズ最適化
import optuna.integration.lightgbm as ilgbm

In [2]:
df = pd.read_csv('..//Data-science//data//boston.csv')

X = df[['INDUS', 'RM', 'TAX', 'PTRATIO', 'LSTAT']]
y = df['house prices']

X_train_val, X_test, y_train_val, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=53)

X_train_val = X_train_val.values
X_train = X_train.values
X_val = X_val.values
X_test = X_test.values

y_train_val = y_train_val.values
y_train = y_train.values
y_val = y_val.values
y_test = y_test.values

### ベイズ最適化

In [3]:
model = lgb.LGBMRegressor()

param_distributions = {
    'n_estimators':optuna.distributions.IntLogUniformDistribution(1e+0, 1e+3, 1),
    'max_depth': optuna.distributions.IntUniformDistribution(1, 20, 1),
    'reg_alpha': optuna.distributions.IntLogUniformDistribution(1e+0, 10.0, 0.1),
    'reg_lambda': optuna.distributions.IntLogUniformDistribution(1e+0, 10.0, 0.1),
    #'subsample_freq':optuna.distributions.IntUniformDistribution(1, 200, 5),
    'min_child_samples':optuna.distributions.IntUniformDistribution(1, 100, 5),
    #'bagging_freq':optuna.distributions.IntUniformDistribution(1, 10, 1),
    'num_leaves': optuna.distributions.IntUniformDistribution(100, 200, 5),
}

# クロスバリデーションによる、ハイパーパラメータの探索設定.cvのデフォルト設定値は5
optuna_search = optuna.integration.OptunaSearchCV(model,param_distributions,cv=5, n_jobs=3,n_trials=25,random_state=42,scoring='neg_root_mean_squared_error')

# 探索の実行
optuna_search.fit(X_train_val, y_train_val)

/home/bi18056/ml/venv/lib/python3.8/site-packages/optuna/distributions.py:359: FutureWarning: Samplers and other components in Optuna will assume that `step` is 1. `step` argument is deprecated and will be removed in the future. The removal of this feature is currently scheduled for v4.0.0, but this schedule is subject to change.
  warnings.warn(
/home/bi18056/ml/venv/lib/python3.8/site-packages/optuna/distributions.py:560: UserWarning: The distribution is specified by [1, 100] and step=5, but the range is not divisible by `step`. It will be replaced by [1, 96].
  warnings.warn(
/tmp/ipykernel_615/2231819116.py:15: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = optuna.integration.OptunaSearchCV(model,param_distributions,cv=5, n_jobs=3,n_trials=25,random_state=42,scoring='neg_root_mean_squared_error')
[I 2022-01-09 15:39:29,894] A new study created in memory with name: no-name-fcfda2da-9ffe-4bbd-880

OptunaSearchCV(estimator=LGBMRegressor(), n_jobs=3, n_trials=25,
               param_distributions={'max_depth': IntUniformDistribution(high=20, low=1, step=1),
                                    'min_child_samples': IntUniformDistribution(high=96, low=1, step=5),
                                    'n_estimators': IntLogUniformDistribution(high=1000.0, low=1.0, step=1),
                                    'num_leaves': IntUniformDistribution(high=200, low=100, step=5),
                                    'reg_alpha': IntLogUniformDistribution(high=10.0, low=1.0, step=0.1),
                                    'reg_lambda': IntLogUniformDistribution(high=10.0, low=1.0, step=0.1)},
               random_state=42, scoring='neg_root_mean_squared_error')

In [4]:
best_params = optuna_search.best_params_
best_params

{'n_estimators': 79,
 'max_depth': 7,
 'reg_alpha': 5,
 'reg_lambda': 3,
 'min_child_samples': 1,
 'num_leaves': 115}

In [5]:
optuna_search.best_params_

{'n_estimators': 79,
 'max_depth': 7,
 'reg_alpha': 5,
 'reg_lambda': 3,
 'min_child_samples': 1,
 'num_leaves': 115}

### パイプライン構築

In [6]:
pipe = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('regressor', lgb.LGBMRegressor(
            objective="regression",
            n_estimators= 68,
            max_depth=11,
            reg_alpha=2,
            reg_lambda=4,
            subsample_freq=151,
            min_child_samples=1,
            bagging_freq=96,
            num_leaves=160
        ))
    ]
)

In [7]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('regressor',
                 LGBMRegressor(bagging_freq=96, max_depth=11,
                               min_child_samples=1, n_estimators=68,
                               num_leaves=160, objective='regression',
                               reg_alpha=2, reg_lambda=4,
                               subsample_freq=151))])

In [8]:
pipe.get_params

<bound method Pipeline.get_params of Pipeline(steps=[('scaler', StandardScaler()),
                ('regressor',
                 LGBMRegressor(bagging_freq=96, max_depth=11,
                               min_child_samples=1, n_estimators=68,
                               num_leaves=160, objective='regression',
                               reg_alpha=2, reg_lambda=4,
                               subsample_freq=151))])>

In [9]:
# r2
pipe.score(X_test, y_test)

0.804943945266863

In [10]:
pred = pipe.predict(X_test)

In [11]:
rmse = np.sqrt(mean_squared_error(y_test, pred))

print(rmse)

3.782089161020395


In [12]:
# モデルの保存
with open('3_pipe_lgbm.model', 'wb') as f:
  pickle.dump(pipe, f)

### 課題3の精度
- rmse: 3.752752469634236
- r2: 0.7988901108422742

## ベイズ最適化1

In [18]:
model = lgb.LGBMRegressor()

param_distributions = {
    'n_estimators':optuna.distributions.IntLogUniformDistribution(1e+0, 1e+3, 1),
    'max_depth': optuna.distributions.IntUniformDistribution(1, 20, 1)
}

# クロスバリデーションによる、ハイパーパラメータの探索設定.cvのデフォルト設定値は5
optuna_search = optuna.integration.OptunaSearchCV(
    model,
    param_distributions,
    cv=5,
    n_jobs=-1,
    n_trials=100,
    random_state=42,
    scoring='neg_root_mean_squared_error')

# 探索の実行
optuna_search.fit(X_train_val, y_train_val)

/tmp/ipykernel_615/2824580095.py:9: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = optuna.integration.OptunaSearchCV(
[I 2022-01-09 15:47:11,207] A new study created in memory with name: no-name-090479d8-2598-409f-9688-14a1f8474661
/home/bi18056/ml/venv/lib/python3.8/site-packages/optuna/study/study.py:393: FutureWarning: `n_jobs` argument has been deprecated in v2.7.0. This feature will be removed in v4.0.0. See https://github.com/optuna/optuna/releases/tag/v2.7.0.
  warnings.warn(
[I 2022-01-09 15:47:11,467] Trial 1 finished with value: -8.627795117318598 and parameters: {'n_estimators': 1, 'max_depth': 20}. Best is trial 1 with value: -8.627795117318598.
[I 2022-01-09 15:47:11,475] Trial 5 finished with value: -8.5306064131743 and parameters: {'n_estimators': 2, 'max_depth': 1}. Best is trial 5 with value: -8.5306064131743.
[I 2022-01-09 15:47:11,554] Trial 3 finished with value: -8.038342766329

OptunaSearchCV(estimator=LGBMRegressor(), n_jobs=-1, n_trials=100,
               param_distributions={'max_depth': IntUniformDistribution(high=20, low=1, step=1),
                                    'n_estimators': IntLogUniformDistribution(high=1000.0, low=1.0, step=1)},
               random_state=42, scoring='neg_root_mean_squared_error')

In [19]:
optuna_search.best_score_

-3.7912583934094153

In [20]:
print(optuna_search.score(X_test, y_test))

-3.9127614133880844


In [21]:
best_params = optuna_search.best_params_
best_params

{'n_estimators': 949, 'max_depth': 2}

In [22]:
from sklearn.decomposition import PCA
from lightgbm.basic import param_dict_to_str

pca = PCA()
regressor = lgb.LGBMRegressor(param_dict_to_str(best_params))

estimators = [
    #('StandScaler', scaler),
    #('feature_selection', fr),
    ('pca', pca),
    ('estimator', regressor)
]

In [23]:
pipe = Pipeline(steps=estimators)

In [24]:
# 学習
pipe.fit(X_train, y_train)

# 予測
pred = pipe.predict(X_test)

[LightGBM] [Warning] Unknown parameter boosting_type=n_estimators=949
[LightGBM] [Warning] max_depth is set=2, max_depth=-1 will be ignored. Current value: max_depth=2


In [25]:
# r2
r2 = pipe.score(X_test, y_test)
print(f'r2: {r2}')

# rmse
rmse = np.sqrt(mean_squared_error(y_test, pred))

print(rmse)

r2: 0.7890087093274946
3.9335466895531046


### ベイズ最適化2

In [ ]:
params = {
        "objective": "regression",
        "metric": "rmse",
        "verbosity": -1,
        "random_seed": 42,
        "boosting_type": "gbdt",
    }

In [ ]:
dtrain = lgb.Dataset(X_train, label=y_train)

tuner = ilgbm.LightGBMTunerCV(
    params,
    dtrain,
    folds=KFold(n_splits=5),
    verbose_eval=100,
    early_stopping_rounds=100,
)

# ハイパーパラメータ探索の実行
tuner.run()

[I 2021-12-30 21:54:50,376] A new study created in memory with name: no-name-db78b237-09c1-4a00-8264-e8212c3dbb4d
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]/home/bi18056/ml/venv/lib/python3.8/site-packages/lightgbm/engine.py:577: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/bi18056/ml/venv/lib/python3.8/site-packages/lightgbm/engine.py:620: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	cv_agg's rmse: 3.89765 + 0.491751
[200]	cv_agg's rmse: 3.87197 + 0.50453


feature_fraction, val_score: 3.864767:  14%|#4        | 1/7 [00:07<00:44,  7.41s/it]

[300]	cv_agg's rmse: 3.87796 + 0.483789


/home/bi18056/ml/venv/lib/python3.8/site-packages/lightgbm/engine.py:577: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/bi18056/ml/venv/lib/python3.8/site-packages/lightgbm/engine.py:620: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


...

[100]	cv_agg's rmse: 3.87261 + 0.444554


min_data_in_leaf, val_score: 3.745051:  80%|########  | 4/5 [00:03<00:00,  1.06it/s]/home/bi18056/ml/venv/lib/python3.8/site-packages/lightgbm/engine.py:577: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/bi18056/ml/venv/lib/python3.8/site-packages/lightgbm/engine.py:620: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	cv_agg's rmse: 4.91372 + 0.45545
[200]	cv_agg's rmse: 4.74367 + 0.462594
[300]	cv_agg's rmse: 4.6469 + 0.441924
[400]	cv_agg's rmse: 4.56992 + 0.417832
[500]	cv_agg's rmse: 4.53881 + 0.423919
[600]	cv_agg's rmse: 4.48926 + 0.384294
[700]	cv_agg's rmse: 4.48542 + 0.343205


min_data_in_leaf, val_score: 3.745051: 100%|##########| 5/5 [00:19<00:00,  3.96s/it]

[800]	cv_agg's rmse: 4.49427 + 0.352493


In [ ]:
best_params = tuner.best_params
best_params

{'objective': 'regression',
 'metric': 'rmse',
 'verbosity': -1,
 'random_seed': 0,
 'boosting_type': 'gbdt',
 'feature_pre_filter': False,
 'lambda_l1': 0.821964022476991,
 'lambda_l2': 0.0004011295898825152,
 'num_leaves': 31,
 'feature_fraction': 0.4,
 'bagging_fraction': 0.7260348407611517,
 'bagging_freq': 5,
 'min_child_samples': 10}